# Лабораторная работа 8. Анализ одноканальной системы массового обслуживания с учетом приоритета заявок
Выполнил студент РТ5-81Б Пакало Александр Сергеевич

Задание 1.
Система электронного документооборота с автоматическим классификатором
поступающей корреспонденции должна учитывать различную важность документов,
количество типов документов $P$,
средняя интенсивность поступления заявок
$\lambda_1, \lambda_2, \dots, \lambda_P$,
средняя интенсивность обслуживания
$\mu_1, \mu_2, \dots, \mu_P$ соответственно, очередь неограничена.
Приоритеты заявок
$p_1 < p_2 < \dots < p_P$.

Оценить среднее время нахождения каждого типа заявки в очереди (теоретически
и экспериментально) для одного из 4 случаев:

- относительный фиксированный приоритет (V1);
- абсолютный фиксированный приоритет (V2);
- приоритет с зависимой задержкой без прерывания (V3);
- приоритет с зависимой задержкой с прерываниями (V4);

в соответствии с вариантом.

In [1]:
Variant <- 5
set.seed(Variant)
V <- sample(c("V1", "V2", "V3", "V4"), 1)
P <- sample(c(4, 6), 1)
if ((V == "V3") | (V == "V4")) {
    b <- sort(sample(c(1:10), P))
}
lambda <- runif(P)
mu <- runif(P, 1, 3)
View(data.frame(P, V))
if ((V == "V3") | (V == "V4")) {
    View(data.frame(lambda, mu, b))
}
if ((V == "V1") | (V == "V2")) {
    View(data.frame(lambda, mu))
}

P,V
<dbl>,<chr>
4,V2


lambda,mu
<dbl>,<dbl>
0.9168758,2.055920
0.2843995,2.615870
0.1046501,2.913000
0.7010575,1.220906


In [2]:
sum(lambda / mu)

[1] 1.164825

Как видно, суммарная нагрузка на систему превышает 1, следовательно будет
накапливаться бесконечная очередь. Немного изменим
исходные данные.

In [3]:
lambda[1] <- 0.1
lambda[2] <- 0.1
ro <- sum(lambda / mu)
ro

[1] 0.6970042

### Теоретически
$$\lambda=\sum_{p=1}^P\lambda_p$$

In [4]:
lambda_ <- sum(lambda)
lambda_

[1] 1.005708

$$\frac{1}{\mu}=\sum_{p=1}^P \frac{\lambda_p}{\lambda}\cdot \frac{1}{\mu_p}$$

In [5]:
ro <- lambda / mu

mu_ <- lambda_ * sum(ro)
mu_

[1] 0.7009825

$$\rho=\frac{\lambda}{\mu}$$

In [6]:
ro_ <- lambda_ / mu_
ro_

[1] 1.434711

$$W_0=\sum_{p=1}^P\frac{\rho_p}{\mu_p}$$

In [7]:
W0 <- sum(ro / mu)
W0

[1] 0.5209205

Для получения $W_p$ воспользуемся формулой

$$W_p= \frac{\frac{\rho_p}{\mu_p}+\sum_{i=p+1}^P \rho_i\cdot \left(\frac{1}{\mu_p}+\frac{1}{\mu_i}\right)+\sum_{i=p+1}^P\rho_i\cdot W_i}{1-\sum_{i=p}^P \rho_i}$$

In [8]:
get_Wqueue <- function(p) {
    num <- ro[p] / mu[p]
    den <- 1 - sum(ro[p:P])

    if (p == P) {
        return(num / den)
    }


    add1 <- sum(unlist(lapply(c((p + 1):P), function(i) ro[i] * (1 / mu[p] + 1 / mu[i]))))
    add2 <- sum(unlist(lapply(c((p + 1):P), function(i) ro[i] * get_Wqueue(i))))

    result <- (num + add1 + add2) / den

    if (result < 0) {
        return(Inf)
    }

    return(result)
}

results <- data.frame(theoretical = unlist(lapply(1:P, get_Wqueue)))
row.names(results) <- 1:P
results

,theoretical
,<dbl>
1,5.786096
2,4.225534
3,3.370473
4,1.104573


### Численно

In [41]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [60]:
create_trajectory <- function(mu, name) {
    return(trajectory(name) %>%
        seize("server", 1) %>%
        timeout(function() rexp(1, mu)) %>%
        release("server", 1))
}

create_application_generator <- function(env, i) {
    name <- gsub(" ", "", paste("documents", i))

    env %>%
        add_generator(
            name_prefix = name,
            trajectory = create_trajectory(mu[i], name),
            distribution = function() rexp(1, lambda[i]),
            priority = i,
            preemptible = i + 1,
        )
}

add_generators <- function(env, n) {
    for (i in 1:n) {
        create_application_generator(env, i)
    }
}

In [43]:
SIMULATION_TIME <- 10000

env %>%
    add_resource("server", preemptive = TRUE)

add_generators(env, P)

env %>% run(until = SIMULATION_TIME)

simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }

simmer environment: SuperDuperSim | now: 10000 | next: 10000.2148647009
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: Generator1_ | monitored: 1 | n_generated: 1047 }
{ Source: Generator2_ | monitored: 1 | n_generated: 949 }
{ Source: Generator3_ | monitored: 1 | n_generated: 1012 }
{ Source: Generator4_ | monitored: 1 | n_generated: 7001 }

In [44]:
programmers <- trajectory("programmers' path") %>%
    ## add an intake activity
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t2)) %>%
    release("server", 1)

In [45]:
arrivals <- env %>% get_mon_arrivals(ongoing = TRUE) # Show ongoing tasks too.
resources <- env %>% get_mon_resources()
arrivals
resources

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
Generator4_0,0.7443853,1.279765,0.53537991,TRUE,1
Generator1_0,2.8573548,4.594245,1.73689045,TRUE,1
Generator1_1,4.1741940,5.583334,0.98908923,TRUE,1
Generator3_0,5.8921775,5.927455,0.03527737,TRUE,1
Generator4_1,7.9792124,8.564903,0.58569086,TRUE,1
Generator4_2,8.1434113,8.582243,0.01733983,TRUE,1
Generator3_1,8.5107356,9.190093,0.60785018,TRUE,1
Generator1_2,9.1699556,9.680157,0.49006382,TRUE,1
Generator4_3,9.9996312,10.274713,0.27508225,TRUE,1


resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,0.7443853,1,0,1,Inf,1,Inf,1
server,1.2797652,0,0,1,Inf,0,Inf,1
server,2.8573548,1,0,1,Inf,1,Inf,1
server,4.1741940,1,1,1,Inf,2,Inf,1
server,4.5942453,1,0,1,Inf,1,Inf,1
server,5.5833345,0,0,1,Inf,0,Inf,1
server,5.8921775,1,0,1,Inf,1,Inf,1
server,5.9274548,0,0,1,Inf,0,Inf,1
server,7.9792124,1,0,1,Inf,1,Inf,1


Отфильтруем NA значения:

In [46]:
arrivals["end_time"][arrivals["start_time"] == -1] <- SIMULATION_TIME
arrivals["start_time"][arrivals["start_time"] == -1] <- SIMULATION_TIME
arrivals["end_time"][is.na(arrivals["end_time"])] <- SIMULATION_TIME
arrivals["activity_time"][is.na(arrivals["activity_time"])] <- 0

Разделим заявки по приоритету и подсчитаем для каждого набора данных время:

In [61]:
patterns <- lapply(1:P, function(i) paste("documents", 1))

results$practical <- lapply(A, function(pattern) {
    arrivals[grepl(arrivals$name, pattern = pattern, fixed = TRUE), ] %>%
        with(mean(end_time - activity_time - start_time))
})
results

,theoretical,practical
,<dbl>,<list>
1,5.786096,6.279072
2,4.225534,5.36344
3,3.370473,3.428539
4,1.104573,1.157447


### Вывод
Как видно, теоретически вычисленное значение с некоторой точностью совпадает
со значением, полученным теоретически. При увеличении количества
экспериментов 𝑁 точность только увеличивается.